In [48]:
# import os
# from dotenv import load_dotenv

# load_dotenv()

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# MODEL = "gpt-3.5-turbo"
MODEL = "mistral"

In [49]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)
# model.invoke("Tell me a joke")

In [50]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
# chain.invoke("Tell me a joke")

In [51]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Allgemeine Geschäftsbedingungen SUS gmbh.pdf")
pages = loader.load_and_split()

In [52]:
from langchain.prompts import PromptTemplate

template = """" \
"Answer the question based on the context below. If you can't answer the question, reply "I don't know"." \
"" \
"Context: {context}" \
"" \
"Question: {question}" \
"""

prompt = PromptTemplate.from_template(template)
# prompt.format(context="Here is some context", question="Here is a question")

In [53]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [54]:
retriever = vectorstore.as_retriever()

In [ ]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

chain.invoke({"question": "Tell me about the Payment terms in this AGB."})

' The payment terms in this Agreement state the following:\n\n1) Payments are due within 7 days after the invoice is issued.\n2) The provider reserves the right to charge a processing fee, or "Bearbeitungsgebühr", equal to 75% of the invoiced amount if payment is not made on time.\n3) If payment is not received within the due date, a monthly penalty fee, or "Strafgebühr", of 25% of the outstanding amount will be charged.'